# 토큰화

토큰화(tokenization)는 자연어를 **모델이 이해할 수 있는 또는 모델이 다룰 수있는 기본 단위(Token)** 분할하는 과정을 말한다.   
토큰으로 나누는 단위는 설계에 따라 문장, 어절, 형태소, 서브워드, 문자, 자모/알파벳 등 다양한 방식으로 나눌 수 있다.   
- 예
```bash
원문: "자연어 처리는 재미있다"
토큰화: ["자연어", "처리", "는", "재미있다"]
```

## 토큰화 방식
- **단어 기반 토큰화(Word-Level Tokenization)**
    - 어절(공백으로 구분) 또는 형태소 단위로 단어를 나누는 전통적인 방식이다.
    - **한국어**는 교착어로 하나의 단어에 다양한 조사/어미가 결합된다. 그래서 어절단위로 토큰화할 경우 어휘사전의 크기가 기하급수적으로 늘어나는 문제가 있다.
      - 예) "학교", "학교가", "학교를", "학교에", "학교에서", "학교로", "학교의", ...
    - 이로 인해 미등록어휘(OOV - Out of Vocabulary)의 증가, 같은 의미를 가지는 단어들이 Vocab에 중복 등록, 메모리 낭비, 학습효율성 저하 등 다양한 문제가 생긴다.
    - 그래서 **한국어의 경우 형태소 단위 토큰화**가 필요하다.

- **서브워드 기반(Subword-level) — BPE, WordPiece, Unigram**
    - Transformer 기반 모델(BERT, GPT, LLaMA 등)에서 표준으로 사용하는 방식.
    - 단어를 기준으로 토큰화하지 않고 **문자(character)와 단어(word)의 중간 수준인 서브워드(subword) 단위로 토큰화**한다.
    - **동작 원리**:
        - 자주 등장하는 문자열 조합(서브워드)을 하나의 토큰으로 구성한다.
        - 빈도가 높은 단어는 하나의 토큰으로, 빈도가 낮거나 희귀한 단어는 여러 서브워드로 분할한다.
    - **예시**:
        ```bash
        입력: "나는 밥을 먹었습니다. 나는 어제 밥을 했습니다."
        
        서브워드 토큰화 결과 (예시):
        ["나는", "밥", "을", "먹", "었", "습니다", ".", "나는", "어제", "밥", "을", "하", "었", "습니다", "."]
        ```
    - **장점**:
        - **미등록 단어(OOV) 문제 해결**: 모든 단어를 서브워드 조합으로 표현 가능
        - **어휘 사전 크기 최적화**: 단어 단위보다 작고, 문자 단위보다 효율적
        - **다국어 지원**: 언어에 구애받지 않는 범용적 토큰화
        - **형태론적 의미 포착**: 접두사, 접미사 등의 의미를 학습 가능
    - **주요 알고리즘**:
        - **BPE (Byte Pair Encoding)**: 가장 빈번한 연속 바이트/문자 쌍을 반복적으로 병합
        - **WordPiece**: BERT에서 사용, BPE와 유사하지만 likelihood 기반으로 병합
        - **Unigram**: 확률 모델 기반으로 최적의 서브워드 분할 선택


# 한국어 형태소 분석기

- kiwipiepy와 konlpy 는 대표적인 한국어 형태소 분석기이다.

## kiwipiepy
**kiwipiepy**는 C++로 구현된 한국어 형태소 분석기 Kiwi(Korean Intelligent Word Identifier)를 Python 환경에서 사용할 수 있도록 한 라이브러리이다. 

- 빠른 속도  
- 최신 품사 체계 지원  
- 사용자 사전 확장 용이  
- 최근 가장 널리 쓰이는 한국어 토크나이저 중 하나이다.
- https://github.com/bab2min/kiwipiepy
  
### 설치 방법

```bash
pip install kiwipiepy
```


In [ ]:
#uv pip install ipykernel ipywidgets
import kiwipiepy

### 주요 클래스 및 함수

#### Kiwi 클래스
- Kiwi의 핵심 클래스이며, 형태소 분석과 토큰화 기능을 모두 제공한다.
- Kiwi 품사는 세종 말뭉치를 기반으로 한다.
  - 품사 시작 글자
  - 체언(명사, 대명사): `N`, 용언(동사, 형용사): `V`, 수식언(관형사, 부사): `M`,  관계언(조사):`J`, 어미: `E`, 기호: `S`
    - https://github.com/bab2min/kiwipiepy?tab=readme-ov-file#%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8
- 메소드
  - `tokenzie(text)`: 형태소 분석 기반 토큰화 수행
  - `analyze(text)`: tokenize보다 좀 더 상세한 분석을 진행한다. 여러 분석결과를 조회할 수있다.
  - `add_user_word(word, pos, score)`: 사전에 직접 단어 등록
  - `space(text)`: 띄어 쓰기 교정

In [ ]:
! uv pip uninstall kiwipiepy

In [ ]:
!uv pip install kiwipiepy==0.21.0

In [ ]:
from kiwipiepy import Kiwi
from pprint import pprint #자료구조를 출력을 보기좋게 print 해줌

In [ ]:
kiwi = Kiwi()

In [ ]:
###################
# 토큰화 = tokenize()
#######################

text = '나는 자연어 처리를 공부한다. \n내일은 뭘 공부할까?'
tokens = kiwi.tokenize(text)

tokens

In [ ]:
for token in tokens:
    r = f'토큰문자열 : {token.form} , 원형(lemma) : {token.lemma}, 품사(tag): {token.tag}\
        시작위치: {token.start}, 글자수: {token.len}, 토큰이 있는 행번호: {token.line_number}\
        몇번째 문장에 있는지: {token.sent_position}, 문장에서 몇번째 어절인지: {token.word_position}'
    print(r)

In [ ]:
#여러 문서를 토큰화 할 때는 list로 묶어서 전달.
text_list = ['나는 자연어 처리를 공부한다.','내일은 뭘 공부할까?']

tokens = kiwi.tokenize(text_list)

tokens

In [ ]:
for token in tokens:
    print(token)

In [ ]:
#여러 문서를 토큰화 할 때는 list로 묶어서 전달.
text = ['나는 자연어 처리를 공부한다.','자연어 처리는 nlp 라고 한다.','내일은 뭘 공부할까?']

result = kiwi.analyze(text , top_n=2)

for token in result:
    pprint(token)

In [ ]:
###################
# 띄어쓰기 교정 = space()
#######################

text = '자연어처리는재미있는분야이다.또재미있는것은무가있을까?'
result = kiwi.space(text)
result

In [ ]:
text_list = ['자연어처리는재미있는분야이다.또재미있는것은무가있을까?','아버지가방에들어가신다']
result = kiwi.space(text_list)
for token in result:
    pprint(token)

In [ ]:
###################
# 문장분리 = split_into_sents()
#######################
txt='특히 양현준은 팀이 0-1로 뒤진 전반 31분 귀중한 동점골을 터뜨렸다.하타테가 올린 크로스를 오른발 슈팅으로 연결해 페예노르트 골문을 연 열었다.오현규와 양현준은 득점, 설영우는 도움을 올렸다.'
result=kiwi.split_into_sents(txt)
result

In [ ]:
for sent in result:
    pprint(sent)

In [ ]:
txt="어제 친구가 '집에가고 싶다.'라고 이야기 했다."
result = kiwi.split_into_sents(txt)
for sent in result:
    pprint(sent)

In [ ]:
sent = result[0]
print(sent.text)

if sent.subs != None:
    print(sent.subs[0].text)

In [ ]:
###################
# 사전에 사용자단어 추가 = add_user_word(단어,품사,score)
# score : 토큰화할 때 그 단어의 운선순위를 조절하는 가중치 값. 클 수록 더 선호하게 된다.
# 0 : 중립 , 고유명사들은 0을 지정한다.
# - 딥러닝 : 5 , 딥 : 10 , 러닝: 10 -> 딥,러닝 으로 나누는 것을 더 선호하게 된다.
#######################
text = '박새로이가 왔다'
kiwi.tokenize(text)

In [ ]:
kiwi.add_user_word('박새로이','NP',0)
kiwi.tokenize(text)

In [ ]:
###################
# 불용어(Stop words) 처리 - 토큰화 했을 떄 제거할 토큰(단어)들. 
#######################

# kiwi 제공하는 불용어
from kiwipiepy.utils import Stopwords

sw = Stopwords()

In [ ]:
# 불용어 조회
# sw.stopwords
len(sw.stopwords)

In [ ]:
text = '나는 자연어 처리를 공부한다. 자연어처리는 NLP라고 한다.'
result = kiwi.tokenize(text)
result

In [ ]:
# stopwords 들 제거 - stopwords객체.filter(토큰list)
result2 = sw.filter(result)
result2

In [ ]:
#불용어 추가/삭제
#sw.add(('단어','품사tag')) # 품사 tag 생략 -> NNP(고유명사)
# sw.remove(('단어','품사tag'))

In [ ]:
result = kiwi.tokenize('이름이 박새로이입니다.')
print(result)
sw.filter(result)

In [ ]:
sw.add(('박새로이','NP'))
sw.filter(result)

In [ ]:
sw.remove(('박새로이','NP'))
sw.filter(result)

In [ ]:
# 명사만 추출
result = kiwi.tokenize('나는 NLP 공부를 어제부터 시작했습니다.')
token_list=[]
for token in result:
    if token.tag.startswith('N'): #명사는 N으로 시작.
        token_list.append(token)

token_list

## KoNLPy(코엔엘파이)
- KoNLPY는 한국어 자연어 처리(Natural Language Processing) 파이썬 라이브러리이다.  한국어 처리를 위한 tokenize, 형태소 분석, 어간추출, 품사부착(POS Tagging) 등의 기능을 제공한다. 
- http://KoNLPy.org/ko/latest/
- 기존의 개발된 다양한 형태소 분석기를 통합해서 동일한 interface로 호출 할 수 있게 해준다.

### KoNLPy 설치
- 설치 순서
  1. Java 실행환경 설치
  2. JPype1 설치
  3. koNLPy 설치

1. **Java 설치**
  - https://www.oracle.com/java/technologies/downloads/
    - OS에 맞는 설치 버전을 다운받아 설치한다.
    - MAC: ARM일 경우: **ARM64 CPU** - ARM64 DMG Installer, **Intel CPU**: x64 DMG Installer
  - 시스템 환경변수 설정
      - `JAVA_HOME` : 설치 경로 지정
      - `Path` : `설치경로\bin` 경로 지정

2. **JPype1 설치**
   - 파이썬에서 자바 모듈을 호출하기 위한 연동 패키지
   - 설치: `pip install JPype1`

3. **KoNLPy 설치**
- `pip install konlpy`

In [ ]:
import konlpy

### 형태소 분석기/사전
- 형태소 사전을 내장하고 있으며 형태소 분석 함수들을 제공하는 모듈

#### KoNLPy 제공 형태소 분석기
- Open Korean Text
    - 트위터에서 개발
    - https://github.com/open-korean-text/open-korean-text
- Hannanum(한나눔)
    - KAIST Semantic Web Research Center 에서 개발
    - http://semanticweb.kaist.ac.kr/hannanum/
- Kkma(꼬꼬마)
    - 서울대학교 IDS(Intelligent Data Systems) 연구실 개발.
    - http://kkma.snu.ac.kr/
- Komoran(코모란)
    - Shineware에서 개발.
    - 오픈소스버전과 유료버전이 있음
    - https://github.com/shin285/KOMORAN
- Mecab(메카브) 
    - 일본어용 형태소 분석기를 한국에서 사용할 수 있도록 수정
    - windows에서는 설치가 안됨
    - https://bitbucket.org/eunjeon/mecab-ko


### 형태소 분석기 공통 메소드
- `morphs(string)` : 형태소 단위로 토큰화(tokenize)
- `nouns(string)` : 명사만 추출하여 토큰화(tokenize)    
- `pos(string)`: 품사 부착
    - 형태소 분석기 마다 사용하는 품사태그가 다르다.
        - https://konlpy-ko.readthedocs.io/ko/v0.5.2/morph/
- `tagset`: 형태소 분석기가 사용하는 품사태그 설명하는 속성. 

# WordCloud

WordCloud는 텍스트 데이터에서 단어의 등장 빈도를 시각적으로 표현한 그래픽이다.
- 특징
  - 자주 등장하는 단어일수록 글자 크기가 커진다.
  - 텍스트 전체의 주제를 직관적으로 파악할 수 있다.
  - 문서의 핵심키워드를 빠르게 파악할 수 있어 텍스트 분석에서 탐색적 데이터 분석(EDA) 단계에서 자주 활용된다.
- 설치
  - `pip install wordcloud`